### Key interactions to json

This notebook converts the key interactions information obtained in csv to json format for subsequent comparison.

In [ ]:
import pandas as pd

df = pd.read_csv('./keyinteractions.csv')
df

In [ ]:
def parse_rule(rule, target):

    def add_prefix(r, target):
        upper = r.upper()
        pref = 'A_' if target != 'MEK1' else 'B_' # MEK1 in chain B
        if "'" in upper: # NAMPT in PPI surface
            upper = upper.replace("'", "")
            pref = 'B_'
        return pref + upper

    rules = []
    for sub_rule in rule.split(';'):
        if '+' in sub_rule:
            res = sub_rule.split('+')
            match_mode = 'all'
        elif '/' in sub_rule:
            res = sub_rule.split('/')
            match_mode = 'any'
        else:
            res = [sub_rule]
            match_mode = 'all'
        rules.append({
            'residues': [add_prefix(r, target) for r in res],
            'match_mode': match_mode
        })
    return rules

In [ ]:
inters = {}
for index, row in df.iterrows():
    target = row['Target']
    inters[target] = {
        col: parse_rule(value, target) 
        for col, value in row[1:].items() if not pd.isna(value)}